In [1]:
def getKeywords(freq_threshold = 10, prob_threshold = 0.5, words_per_iter = 5, iters = 3):
    
    sent_tokenized_reviews =  data['Content'].apply(lambda x: x.decode('utf-8')).apply(nltk.tokenize.sent_tokenize)
    sentences = [sentence for review in sent_tokenized_reviews for sentence in review]

    vectorizer = sk.feature_extraction.text.CountVectorizer(min_df = freq_threshold, binary=True,
                                                        ngram_range=(1,1),token_pattern = r'[a-zA-Z]{3,}',
                                                        stop_words=nltk.corpus.stopwords.words('english'))
    X = vectorizer.fit_transform(sentences)
    X = X.toarray() #convert sparse array to array

    seeds =  {"Value" : ["value", "price", "quality","worth"],
              "Rooms" : ["room", "suite", "view", "bed"],
              "Location" : ["location", "traffic", "minute", "restaurant"],
              "Cleanliness" : ["clean", "dirty", "maintain", "smell"],
              "Check in / front desk": ["stuff", "check", "help", "reservation"],
              "Service" : ["service", "food", "breakfast", "buffet"],
              "Business service" : ["business", "center", "computer", "internet"]
             }
    
    count_sents_with_word = np.sum(X,axis=0) # works because binary
    
    # find Probability(sentence(S) has aspect(A) GIVEN S has word(W)) = count(S that have A and have W) / count(S that have W)

    for i in range(iters):
        for aspect in seeds:

            # if condition ensures code runs even if seed word not in our corpus by some chance
            seed_indices = [vectorizer.vocabulary_[word] for word in seeds[aspect] if vectorizer.vocabulary_.get(word) != None]
            count_sents_with_asp = np.sum(np.logical_or.reduce(X[:,seed_indices].T))
            count_sents_with_word_asp = np.sum(X[np.logical_or.reduce(X[:,seed_indices].T),:],axis=0)

            prob_asp_given_word = count_sents_with_word_asp.astype(float)/count_sents_with_word
            sorted_indices = np.argsort(prob_asp_given_word)[::-1]
            count = 0
            for j in sorted_indices:
                if count > words_per_iter or prob_asp_given_word[j] < prob_threshold:
                    break
                else:
                    if vectorizer.get_feature_names()[j] not in seeds[aspect]:
                        seeds[aspect].append(vectorizer.get_feature_names()[j])
                        count += 1
    return seeds